# 1. Khai báo các thư viện cần thiết.

In [6]:
import logging
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import requests
import json
import codecs

# 2. Hàm ghi dữ liệu vào file data_products.json.

In [7]:
def write_reviews_to_file(_data):
    filename = 'data_products.json'
    with open(filename, "r", encoding='utf-8') as f:
        try:
            data = json.load(f)
        except json.decoder.JSONDecodeError:
            data = []
          
    data.append(_data)

    with codecs.open(filename, "w", encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)

# 3. Hàm lấy đường dẫn đánh giá sản phẩm.

In [8]:
def get_product_review_url(url):
    new_url = url.split("?", 1)[0]
    return f'{new_url}/danh-gia'

# 4. Crawling Data.

In [12]:
headers = {'User-Agent': 'Mozilla/5.0'}
def get_reviews(_product_link):
    response = requests.get(f'{_product_link}', headers=headers)
    if(response.ok):
        soup = BeautifulSoup(response.content, "html.parser")
        product_name = soup.find('div', class_="box-pdt__content").find('h3').text
        data = {"product name": product_name, "reviews": []}
        print(f'Crawling reviews of {product_name}')
        print('*-----------------------------------*')

        PAGE = 1
        while True:
            response = requests.get(f'{_product_link}?p={PAGE}', headers=headers)
            soup = BeautifulSoup(response.content, "html.parser")
            comments = soup.find_all("div", class_="comment__item par")
            if comments:
                for comment in comments:
                    username = comment.find("div", class_="item-top").find("p", class_="txtname").text
                    star = comment.find('div', class_="item-rate").find('div', class_="comment-star").find_all('i', class_='icon-star')
                    content = comment.find('div', class_="comment-content").find("p", class_="cmt-txt").text
                    data['reviews'].append(
                        {"username": username, "content": content, "star": len(star)}
                    )
            else:
                break
            PAGE += 1
        # Lưu ý tạo file tên: "data_products.json" trước khi gọi hàm bên dưới.
        write_reviews_to_file(data)
    

# 5. Hàm khởi tạo trình duyệt.

In [10]:
def init_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(options=options, executable_path="chromedriver.exe") # Đường dẫn đến file chromedriver.exe.

# 6. Main

In [11]:
#@title Main
CATEGORIES = ['dtdd', 'laptop']
for CATEGORY in CATEGORIES:
    browser = init_browser()
    browser.get(f'https://www.thegioididong.com/{CATEGORY}')
    sleep(5)

    while True:
        try:
            view_more_link = browser.find_element_by_xpath("/html/body/section/div[3]/div[2]")
            view_more_link.click()
            sleep(5)
        except Exception as e:
            logging.info(e)
            break

    products = browser.find_elements_by_xpath('/html/body/section/div[3]/ul/li')

    for product in products:
        product_link = product.find_element_by_class_name('main-contain').get_attribute('href')
        sleep(3)
        get_reviews(get_product_review_url(product_link))

    browser.close()
    browser.quit()

Crawling reviews of Samsung Galaxy Z Fold3 5G 256GB
*-----------------------------------*
Crawling reviews of iPhone 12 64GB
*-----------------------------------*
Crawling reviews of Xiaomi Mi 11 Lite
*-----------------------------------*
Crawling reviews of Vivo Y12s (4GB/128GB)
*-----------------------------------*
Crawling reviews of OPPO Reno6 Z 5G
*-----------------------------------*
Crawling reviews of Samsung Galaxy A52 5G
*-----------------------------------*
Crawling reviews of Vivo Y53s
*-----------------------------------*
Crawling reviews of Realme C21Y 4GB
*-----------------------------------*
Crawling reviews of Xiaomi Redmi Note 10 5G 8GB
*-----------------------------------*
Crawling reviews of Vsmart Star 5 (4GB/64GB)
*-----------------------------------*
Crawling reviews of OPPO Reno6 5G
*-----------------------------------*
Crawling reviews of Realme C25s
*-----------------------------------*
Crawling reviews of OPPO A74
*-----------------------------------*
Crawlin